In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from collections import Counter 
from tqdm import tqdm
from random import seed, randint, choice

from swarm_sim import *

%matplotlib inline 

In [2]:
# 1st dataset: N=50
PATH = '..\\..\\data\\swarm-50-sats-scenario\\coords_v1_if_LLO-'
EXPORT_PATH = 'output\\data'
ROW_DATA = 7
NB_NODES = 50
DURATION = 8641 # Number of data rows, not time!
REVOLUTION = 1800 # Number of data rows
CONNECTION_RANGE = 30 # km


In [3]:
satellites = {} # Dict(sat_id: DataFrame)
with tqdm(total=NB_NODES, desc='Extracting data') as pbar:
    for i in range(NB_NODES):
        df_data = pd.read_csv(PATH+str(i)+'.csv', skiprows= lambda x: x<ROW_DATA, header=0)
        satellites[i] = df_data
        pbar.update(1)
        
swarm_data = {} # Dict{timestamp: Swarm}
with tqdm(total=REVOLUTION, desc='Converting to topologies') as pbar:
    for t in range(REVOLUTION):
        swarm_data[t] = Swarm(CONNECTION_RANGE,
                    nodes = [Node(id, sat['xF[km]'].iloc[t], sat['yF[km]'].iloc[t], sat['zF[km]'].iloc[t]) for id,sat in satellites.items()]
                    )
        pbar.update(1)

neighbor_matrices = {} # Dict{timestamp: matrix}
with tqdm(total=REVOLUTION, desc='Computing neighbor matrices') as pbar:
    for t in range(REVOLUTION):
        neighbor_matrices[t] = swarm_data[t].neighbor_matrix()
        pbar.update(1)

topo_graphs = {} # Dict{timestamp: Graph}
with tqdm(total=REVOLUTION, desc='Converting to NetworkX graphs') as pbar:
    for t in range(REVOLUTION):
        topo_graphs[t] = swarm_data[t].swarm_to_nxgraph()
        pbar.update(1)

Extracting data:   0%|          | 0/50 [00:00<?, ?it/s]

Converting to NetworkX graphs: 100%|██████████| 1800/1800 [00:05<00:00, 302.35it/s]


## Temporal evolution
Analysed metrics:
- Network size (NS)
- Average degree (AD): number of direct neighbor(s) for each node, averaged
- Graph density (GD): ratio of observed edges over maximum possible number of edges
- Average clustering coefficient (ACC): ratio of observed edges in the neighborhood over maximum possible number of edges, averaged
- Diameter (Dia): maximum shortest path length between all pairs of nodes

Implemented algorithms for comparison:
- Random Node Division (RND)
- Multiple Independent Random Walks (MIRW)
- Forest Fire Division (FFD)

In [16]:
def jain(avg, std):
    cv = std/avg
    return 1.0/(1 + pow(cv,2))

In [9]:
# Reference temporal evolution: Average Degree, Graph Density, Average Clustering Coefficient
ref_ad, ref_gd, ref_acc = [], [], []

for t,swarm in swarm_data.items():
    ref_ad.append(np.mean(swarm.degree()))
    ref_gd.append(swarm.graph_density())
    ref_acc.append(np.mean(swarm.cluster_coef()))
    
# Run only once
diameters = []
with tqdm(total=len(swarm_data.keys()), desc='Diameter') as pbar:
    for swarm in swarm_data.values():
        diameters.append(swarm.diameter(swarm)[2]-1)
        pbar.update(1)
ref_ml = np.mean(diameters)

Diameter: 100%|██████████| 1800/1800 [02:32<00:00, 11.80it/s]


In [18]:
avg_ref_ad = np.mean(ref_ad)
avg_ref_acc = np.mean(ref_acc)
avg_ref_gd = np.mean(ref_gd)

x = 2
print('Reference values (whole swarm)')
print('\tAverage degree:', round(avg_ref_ad, x))
print('\tAverage clustering coefficient:', round(avg_ref_acc, x))
print('\tGraph Density:', round(avg_ref_gd, x))

Reference values (whole swarm)
	Average degree: 6.53
	Average clustering coefficient: 0.49
	Graph Density: 0.13


In [24]:
NB_GROUPS = 10
NB_REPETITIONS = 1

## 1. Random Node Division

In [5]:
graph = topo_graphs[0]
print(graph)

Graph with 50 nodes and 320 edges


In [4]:
swarm = swarm_data[0]
swarm.reset_groups()

groups = swarm.RND(n=10, s=1, by_id=True) # <==================== ALGO CHOICE 

group_assignment = {}
for node in swarm.nodes:
    group_assignment[node.id] = node.group

In [14]:
# use this to compute group degree (replace nbunch by group nodes)
np.mean(nx.degree(graph, nbunch=[0,1]))

2.25

In [10]:
# Separate groups into swarms
swarm_rnd = {} # Dict(timestamp:swarms)

for t,swarm in swarm_data.items():
    swarms = [] #List(Swarm)
    for gid in range(NB_GROUPS):
        sw = Swarm(connection_range=30000,
                        nodes=[node for node in swarm.nodes if node.group==gid])
        neighbor_matrix = sw.neighbor_matrix()
        swarms.append(sw)
    swarm_rnd[t] = swarms

In [11]:
# Group temporal evolution: Average Degree, Graph Density, Average Clustering Coefficient
group_ad, group_gd, group_acc = [], [], []

for t,swarms in swarm_rnd.items():
    group_ad.append(np.nanmean([np.nanmean(sw.degree()) for sw in swarms]))
    group_gd.append(np.mean([sw.graph_density() for sw in swarms]))
    group_acc.append(np.nanmean([np.nanmean(sw.cluster_coef()) for sw in swarms]))  

avg_group_ad = np.nanmean(group_ad)
avg_group_acc = np.nanmean(group_acc)    
avg_group_gd = np.nanmean(group_gd)

std_ad = np.sqrt(np.nanmean((group_ad - avg_ref_ad)**2))
std_acc = np.sqrt(np.nanmean((group_acc - avg_ref_acc)**2))
std_gd = np.sqrt(np.nanmean((group_gd - avg_ref_gd)**2))

C:\Users\ankoc\AppData\Local\Temp\ipykernel_1732\268300152.py:5: RuntimeWarning: Mean of empty slice
  group_ad.append(np.nanmean([np.nanmean(sw.degree()) for sw in swarms]))
C:\Users\ankoc\AppData\Local\Temp\ipykernel_1732\268300152.py:7: RuntimeWarning: Mean of empty slice
  group_acc.append(np.nanmean([np.nanmean(sw.cluster_coef()) for sw in swarms]))


In [12]:
avg_group_ad = np.mean(group_ad)
avg_group_acc = np.mean(group_acc)    
avg_group_gd = np.mean(group_gd)

std_ad = np.sqrt(np.mean((group_ad - avg_ref_ad)**2))
std_acc = np.sqrt(np.mean((group_acc - avg_ref_acc)**2))
std_gd = np.sqrt(np.mean((group_gd - avg_ref_gd)**2))

print('\nGroup values')
print('\tAverage degree:', round(avg_group_ad, x))
print('\tAverage clustering coefficient:', round(avg_group_acc, x))
print('\tGraph Density:', round(avg_group_gd, x))

print('\nStandard deviations')
print('\tAD:', round(std_ad, x))
print('\tACC:', round(std_acc, x))
print('\tGD:', round(std_gd, x))

print('\nVariation coefficients')
print('\tAD:', round(std_ad/avg_group_ad, x+2))
print('\tACC:', round(std_acc/avg_group_acc, x+2))
print('\tGD:', round(std_gd/avg_group_gd, x+2))

Reference values (whole swarm)
	Average degree: 6.53
	Average clustering coefficient: 0.49
	Graph Density: 0.13

Group values
	Average degree: 10.02
	Average clustering coefficient: 0.52
	Graph Density: 0.96

Standard deviations
	AD: 4.65
	ACC: 0.05
	GD: 0.83

Variation coefficients
	AD: 0.4644
	ACC: 0.1065
	GD: 0.8638


In [14]:
group_diameters = []

with tqdm(total=NB_GROUPS, desc='Group diameter') as pbar:
    for group_id in range(NB_GROUPS):
        for t,swarm in swarm_data.items():
            group_diameters.append(swarm.diameter(swarm_rnd[t][group_id])[2]-1)
        pbar.update(1)
            

Group diameter: 100%|██████████| 10/10 [01:27<00:00,  8.79s/it]


In [22]:
avg_dia = np.mean(group_diameters)
std_dia = np.std(group_diameters)

print('Group diameter')
print('Mean:', round(avg_dia, x))
print('Deviation:', round(std_dia, x))
print('Jains index:', round(jain(avg_dia, std_dia), x+2))

Group diameter
Mean: 3.24
Deviation: 2.08
Jains index: 0.7077


In [23]:
network_sizes = []

for t, group_data in group_distrib.items(): # comm_data = Dict(comm_id:list(count)), groups = Dict(timestamp:comm_data)
    network_sizes.extend([e for distribs in group_data.values() for e in distribs])
    pbar.update()

avg_ns = np.mean(network_sizes)
std_ns = np.std(network_sizes)

print('Group size')
print('Mean:', round(avg_ns, x))
print('Deviation:', round(std_ns, x))
print('Jains index:', round(jain(avg_ns, std_ns), x+2))

Group size
Mean: 5.0
Deviation: 2.12
Jains index: 0.8472
